In [32]:
!pip install pyspark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 823.7 kB/s eta 0:00:00a 0:00:01


In [33]:
import pyspark

In [7]:
import panda 
pd.read_csv('online_retail_combined')

ModuleNotFoundError: No module named 'panda'

In [6]:
!pip install pandas

In [17]:
import pandas as pd

In [34]:
pd.read_csv("/home/jovyan/work/online_retail_combined.csv")

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,fiscal_year
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,2009-2010
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,2009-2010
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,2009-2010
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,2009-2010
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,2009-2010
...,...,...,...,...,...,...,...,...,...
1067366,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,2010-2011
1067367,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,2010-2011
1067368,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,2010-2011
1067369,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France,2010-2011


In [19]:
import os

for root, dirs, files in os.walk("/", topdown=True):
    if "online_retail_combined.csv" in files:
        print("FOUND →", os.path.join(root, "online_retail_combined.csv"))
        break


FOUND → /home/jovyan/work/online_retail_combined.csv


In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Retail ETL Pipeline').getOrCreate()

In [2]:
spark

In [34]:
df_pyspark=spark.read.csv('online_retail_combined.csv')

In [36]:
df_pyspark.show()

+-------+---------+--------------------+--------+-------------------+-----+-----------+--------------+-----------+
|    _c0|      _c1|                 _c2|     _c3|                _c4|  _c5|        _c6|           _c7|        _c8|
+-------+---------+--------------------+--------+-------------------+-----+-----------+--------------+-----------+
|Invoice|StockCode|         Description|Quantity|        InvoiceDate|Price|Customer ID|       Country|fiscal_year|
| 489434|    85048|15CM CHRISTMAS GL...|      12|2009-12-01 07:45:00| 6.95|    13085.0|United Kingdom|  2009-2010|
| 489434|   79323P|  PINK CHERRY LIGHTS|      12|2009-12-01 07:45:00| 6.75|    13085.0|United Kingdom|  2009-2010|
| 489434|   79323W| WHITE CHERRY LIGHTS|      12|2009-12-01 07:45:00| 6.75|    13085.0|United Kingdom|  2009-2010|
| 489434|    22041|"RECORD FRAME 7""...|      48|2009-12-01 07:45:00|  2.1|    13085.0|United Kingdom|  2009-2010|
| 489434|    21232|STRAWBERRY CERAMI...|      24|2009-12-01 07:45:00| 1.25|    1

In [38]:
df_pyspark=spark.read.option('header','true').csv('online_retail_combined.csv')

In [39]:
type(df_pyspark)

pyspark.sql.dataframe.DataFrame

In [41]:
df_pyspark.printSchema()

root
 |-- Invoice: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- fiscal_year: string (nullable = true)



In [42]:
df_pyspark.head(2)

[Row(Invoice='489434', StockCode='85048', Description='15CM CHRISTMAS GLASS BALL 20 LIGHTS', Quantity='12', InvoiceDate='2009-12-01 07:45:00', Price='6.95', Customer ID='13085.0', Country='United Kingdom', fiscal_year='2009-2010'),
 Row(Invoice='489434', StockCode='79323P', Description='PINK CHERRY LIGHTS', Quantity='12', InvoiceDate='2009-12-01 07:45:00', Price='6.75', Customer ID='13085.0', Country='United Kingdom', fiscal_year='2009-2010')]

In [22]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("RetailAnalysis")
    .master("spark://spark-master:7077")
    .config("spark.ui.port", "8084")
    .config("spark.driver.host", "jupyter")
    .config("spark.driver.bindAddress", "0.0.0.0")
    .getOrCreate()
)
print(f"Spark UI is available at: {spark.sparkContext.uiWebUrl}")


Spark UI is available at: http://42ba6cbfebdf:4040


In [20]:
df = spark.range(0, 1_000_000)
df.count()


1000000

In [38]:
spark.stop()

In [58]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from minio import Minio
import pandas as pd
import io

print("=" * 70)
print("🚀 STARTING RETAIL ETL PIPELINE")
print("=" * 70)

# Step 1: Create Spark Session
print("\n[1/9] Creating Spark Session...")

spark = SparkSession.builder \
    .appName("Retail ETL Pipeline") \
    .master("spark://spark-master:7077") \
    .config("spark.driver.host", "jupyter") \
    .config("spark.driver.bindAddress", "0.0.0.0") \
    .config("spark.driver.port", "7078") \
    .config("spark.blockManager.port", "7079") \
    .config("spark.jars.packages", 
            "org.apache.hadoop:hadoop-aws:3.3.4,"
            "com.amazonaws:aws-java-sdk-bundle:1.12.262") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minio") \
    .config("spark.hadoop.fs.s3a.secret.key", "minio123") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.executor.memory", "2g") \
    .config("spark.driver.memory", "2g") \
    .config("spark.cores.max", "1") \
    .getOrCreate()

print("✅ Spark Session Created!")

# Verify connection
print("\n" + "=" * 70)
print("🌐 SPARK UI INFORMATION")
print("=" * 70)
print(f"Spark UI URL: {spark.sparkContext.uiWebUrl}")
print(f"Application ID: {spark.sparkContext.applicationId}")
print(f"Application Name: {spark.sparkContext.appName}")
print(f"Master: {spark.sparkContext.master}")
print("=" * 70)

🚀 STARTING RETAIL ETL PIPELINE

[1/9] Creating Spark Session...
✅ Spark Session Created!

🌐 SPARK UI INFORMATION
Spark UI URL: http://jupyter:4040
Application ID: app-20251221152118-0007
Application Name: Retail ETL Pipeline
Master: spark://spark-master:7077


In [35]:
pd.read_csv("/home/jovyan/work/online_retail_combined.csv")

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,fiscal_year
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,2009-2010
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,2009-2010
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,2009-2010
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,2009-2010
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,2009-2010
...,...,...,...,...,...,...,...,...,...
1067366,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,2010-2011
1067367,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,2010-2011
1067368,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,2010-2011
1067369,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France,2010-2011


In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

# Step 2: Load the data using Spark
print("\n[1/8] Loading raw data...")
df_raw = spark.read.csv("/opt/spark-data/online_retail_combined.csv",
                        header=True,
                        inferSchema=True)

row_count = df_raw.count()
print(f"✅ Loaded {row_count:,} rows")
df_raw.printSchema()


# STEP 1: DATA QUALITY CHECKS
# ============================================================
print("\n[2/8] Running data quality checks...")

# Check for nulls in critical columns
null_counts = df_raw.select([
    count(when(col(c).isNull(), c)).alias(c) 
    for c in df_raw.columns
])
print("\n📊 Null value counts:")
null_counts.show()

# Check for duplicates
total_rows = df_raw.count()
distinct_rows = df_raw.distinct().count()
print(f"\n📋 Total rows: {total_rows:,}")
print(f"📋 Distinct rows: {distinct_rows:,}")
print(f"📋 Duplicates: {total_rows - distinct_rows:,}")

# ============================================================
# STEP 2: DATA CLEANING
# ============================================================
print("\n[3/8] Cleaning data...")

df_cleaned = df_raw \
    .filter(col("Invoice").isNotNull()) \
    .filter(col("StockCode").isNotNull()) \
    .filter(col("Quantity") > 0) \
    .filter(col("Price") > 0) \
    .filter(col("Customer ID").isNotNull()) \
    .dropDuplicates()

print(f"✅ After cleaning: {df_cleaned.count():,} rows")
print(f"   Removed: {df_raw.count() - df_cleaned.count():,} rows")

# ============================================================
# STEP 3: DATA TRANSFORMATION
# ============================================================
print("\n[4/8] Transforming data...")

df_transformed = df_cleaned \
    .withColumn("InvoiceDate", to_timestamp(col("InvoiceDate"), "yyyy-MM-dd HH:mm:ss")) \
    .withColumn("TotalAmount", col("Quantity") * col("Price")) \
    .withColumn("Year", year(col("InvoiceDate"))) \
    .withColumn("Month", month(col("InvoiceDate"))) \
    .withColumn("DayOfWeek", dayofweek(col("InvoiceDate"))) \
    .withColumn("Hour", hour(col("InvoiceDate"))) \
    .withColumn("CustomerID", col("Customer ID").cast("string")) \
    .drop("Customer ID")

print("✅ Added columns: TotalAmount, Year, Month, DayOfWeek, Hour")

# ============================================================
# STEP 4: CREATE BUSINESS METRICS
# ============================================================
print("\n[5/8] Creating business metrics...")

# Customer-level aggregations
df_customer_metrics = df_transformed.groupBy("CustomerID") \
    .agg(
        count("Invoice").alias("TotalOrders"),
        sum("TotalAmount").alias("TotalSpent"),
        avg("TotalAmount").alias("AvgOrderValue"),
        countDistinct("Invoice").alias("UniqueInvoices"),
        min("InvoiceDate").alias("FirstPurchaseDate"),
        max("InvoiceDate").alias("LastPurchaseDate")
    ) \
    .withColumn("DaysSinceFirstPurchase", 
                datediff(col("LastPurchaseDate"), col("FirstPurchaseDate")))

print("✅ Customer metrics created")
df_customer_metrics.show(5)

# Product-level aggregations
df_product_metrics = df_transformed.groupBy("StockCode", "Description") \
    .agg(
        sum("Quantity").alias("TotalQuantitySold"),
        sum("TotalAmount").alias("TotalRevenue"),
        countDistinct("CustomerID").alias("UniqueCustomers"),
        avg("Price").alias("AvgPrice")
    ) \
    .orderBy(desc("TotalRevenue"))

print("\n✅ Product metrics created")
df_product_metrics.show(5)

# ============================================================
# STEP 5: FEATURE ENGINEERING
# ============================================================
print("\n[6/8] Feature engineering...")

# RFM Analysis (Recency, Frequency, Monetary)
from datetime import datetime
max_date = df_transformed.select(max("InvoiceDate")).collect()[0][0]

df_rfm = df_transformed.groupBy("CustomerID") \
    .agg(
        datediff(lit(max_date), max("InvoiceDate")).alias("Recency"),
        countDistinct("Invoice").alias("Frequency"),
        sum("TotalAmount").alias("Monetary")
    )

# Add RFM scores (1-5 scale)
rfm_quantiles = df_rfm.approxQuantile(["Recency", "Frequency", "Monetary"], 
                                       [0.2, 0.4, 0.6, 0.8], 0.01)

df_rfm_scored = df_rfm \
    .withColumn("R_Score", 
                when(col("Recency") <= rfm_quantiles[0][0], 5)
                .when(col("Recency") <= rfm_quantiles[0][1], 4)
                .when(col("Recency") <= rfm_quantiles[0][2], 3)
                .when(col("Recency") <= rfm_quantiles[0][3], 2)
                .otherwise(1)) \
    .withColumn("F_Score",
                when(col("Frequency") >= rfm_quantiles[1][3], 5)
                .when(col("Frequency") >= rfm_quantiles[1][2], 4)
                .when(col("Frequency") >= rfm_quantiles[1][1], 3)
                .when(col("Frequency") >= rfm_quantiles[1][0], 2)
                .otherwise(1)) \
    .withColumn("M_Score",
                when(col("Monetary") >= rfm_quantiles[2][3], 5)
                .when(col("Monetary") >= rfm_quantiles[2][2], 4)
                .when(col("Monetary") >= rfm_quantiles[2][1], 3)
                .when(col("Monetary") >= rfm_quantiles[2][0], 2)
                .otherwise(1)) \
    .withColumn("RFM_Score", 
                concat(col("R_Score"), col("F_Score"), col("M_Score")))

print("✅ RFM analysis complete")
df_rfm_scored.show(10)

# ============================================================
# STEP 6: OUTLIER DETECTION
# ============================================================
print("\n[7/8] Detecting outliers...")

# Find outliers using IQR method
quantiles = df_transformed.approxQuantile("TotalAmount", [0.25, 0.75], 0.01)
Q1, Q3 = quantiles[0], quantiles[1]
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_outliers = df_transformed.filter(
    (col("TotalAmount") < lower_bound) | 
    (col("TotalAmount") > upper_bound)
)

print(f"✅ Found {df_outliers.count():,} outlier transactions")
print(f"   Lower bound: ${lower_bound:.2f}")
print(f"   Upper bound: ${upper_bound:.2f}")

# ============================================================
# ============================================================
# STEP 7: SAVE TO ICEBERG LAKEHOUSE (YOUR CATALOG!)
# ============================================================
print("\n[8/8] Saving to Iceberg Lakehouse...")

try:
    # BRONZE: Raw cleaned data
    df_transformed.writeTo("demo.bronze.retail_transactions") \
        .using("iceberg") \
        .partitionedBy("Year", "Month") \
        .createOrReplace()
    print("✅ BRONZE: demo.bronze.retail_transactions")

    # SILVER: Customer metrics  
    df_customer_metrics.writeTo("demo.silver.customer_metrics") \
        .using("iceberg") \
        .createOrReplace()
    print("✅ SILVER: demo.silver.customer_metrics")

    # SILVER: Product metrics
    df_product_metrics.writeTo("demo.silver.product_metrics") \
        .using("iceberg") \
        .createOrReplace()
    print("✅ SILVER: demo.silver.product_metrics")

    # SILVER: RFM analysis
    df_rfm_scored.writeTo("demo.silver.rfm_analysis") \
        .using("iceberg") \
        .createOrReplace()
    print("✅ SILVER: demo.silver.rfm_analysis")

    # GOLD: Top customers by RFM score
    df_top_customers = df_rfm_scored.filter(col("RFM_Score") >= "555") \
        .select("CustomerID", "Recency", "Frequency", "Monetary", "RFM_Score")
    
    df_top_customers.writeTo("demo.gold.top_customers") \
        .using("iceberg") \
        .createOrReplace()
    print("✅ GOLD: demo.gold.top_customers")

except Exception as e:
    print(f"⚠️  Iceberg save error: {e}")



[1/8] Loading raw data...
✅ Loaded 1,067,371 rows
root
 |-- Invoice: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- Price: double (nullable = true)
 |-- Customer ID: double (nullable = true)
 |-- Country: string (nullable = true)
 |-- fiscal_year: string (nullable = true)


[2/8] Running data quality checks...

📊 Null value counts:
+-------+---------+-----------+--------+-----------+-----+-----------+-------+-----------+
|Invoice|StockCode|Description|Quantity|InvoiceDate|Price|Customer ID|Country|fiscal_year|
+-------+---------+-----------+--------+-----------+-----+-----------+-------+-----------+
|      0|        0|       4382|       0|          0|    0|     243007|      0|          0|
+-------+---------+-----------+--------+-----------+-----+-----------+-------+-----------+


📋 Total rows: 1,067,371
📋 Distinct rows: 1,055,238
📋 D

In [2]:
df_raw = spark.read.csv("/opt/spark-data/online_retail_combined.csv",
                        header=True,
                        inferSchema=True)

df_raw.show(5)


+-------+---------+--------------------+--------+-------------------+-----+-----------+--------------+-----------+
|Invoice|StockCode|         Description|Quantity|        InvoiceDate|Price|Customer ID|       Country|fiscal_year|
+-------+---------+--------------------+--------+-------------------+-----+-----------+--------------+-----------+
| 489434|    85048|15CM CHRISTMAS GL...|      12|2009-12-01 07:45:00| 6.95|    13085.0|United Kingdom|  2009-2010|
| 489434|   79323P|  PINK CHERRY LIGHTS|      12|2009-12-01 07:45:00| 6.75|    13085.0|United Kingdom|  2009-2010|
| 489434|   79323W| WHITE CHERRY LIGHTS|      12|2009-12-01 07:45:00| 6.75|    13085.0|United Kingdom|  2009-2010|
| 489434|    22041|"RECORD FRAME 7""...|      48|2009-12-01 07:45:00|  2.1|    13085.0|United Kingdom|  2009-2010|
| 489434|    21232|STRAWBERRY CERAMI...|      24|2009-12-01 07:45:00| 1.25|    13085.0|United Kingdom|  2009-2010|
+-------+---------+--------------------+--------+-------------------+-----+-----

In [2]:
!pip install Minio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.8/93.8 kB 731.3 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 2.3 MB/s eta 0:00:0000:0100:01


In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from minio import Minio
import io

# Create Spark Session with Iceberg + MinIO
spark = SparkSession.builder \
    .appName("Retail ETL Pipeline") \
    .master("spark://spark-master:7077") \
    .config("spark.sql.extensions",
            "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.iceberg",
            "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.iceberg.catalog-impl",
            "org.apache.iceberg.rest.RESTCatalog") \
    .config("spark.sql.catalog.iceberg.uri",
            "http://iceberg-rest:8181") \
    .config("spark.sql.catalog.iceberg.warehouse",
            "s3a://lakehouse/warehouse") \
    .config("spark.sql.catalog.iceberg.io-impl",
            "org.apache.iceberg.aws.s3.S3FileIO") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minio") \
    .config("spark.hadoop.fs.s3a.secret.key", "minio123") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.hadoop.fs.s3a.impl",
            "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config(
        "spark.jars.packages",
        ",".join([
            "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.0",
            "org.apache.hadoop:hadoop-aws:3.3.4",
            "com.amazonaws:aws-java-sdk-bundle:1.12.262"
        ])
    ) \
    .getOrCreate()

print("✅ Spark session created successfully!")
print(f"Spark version: {spark.version}")
print(f"App name: {spark.sparkContext.appName}")

# Test connection to MinIO
print("\n🔍 Testing MinIO connection...")
try:
    # List catalogs
    catalogs = spark.sql("SHOW CATALOGS").collect()
    print("Available catalogs:")
    for catalog in catalogs:
        print(f"  - {catalog[0]}")
except Exception as e:
    print(f"⚠️  Note: {e}")

print("\n🎯 Spark is ready for ETL!")
print("Next steps:")
print("  1. Create schema: spark.sql('CREATE SCHEMA IF NOT EXISTS iceberg.retail')")
print("  2. Load data from MinIO")
print("  3. Transform and write to Iceberg tables")

✅ Spark session created successfully!
Spark version: 3.5.0
App name: Retail ETL Pipeline

🔍 Testing MinIO connection...
Available catalogs:
  - spark_catalog

🎯 Spark is ready for ETL!
Next steps:
  1. Create schema: spark.sql('CREATE SCHEMA IF NOT EXISTS iceberg.retail')
  2. Load data from MinIO
  3. Transform and write to Iceberg tables


In [9]:
# STEP 2: CREATE ICEBERG SCHEMA (NAMESPACE)
# ============================================================
print("\n[2/10] Creating Iceberg schemas...")

spark.sql("CREATE NAMESPACE IF NOT EXISTS iceberg.bronze")
spark.sql("CREATE NAMESPACE IF NOT EXISTS iceberg.silver")
spark.sql("CREATE NAMESPACE IF NOT EXISTS iceberg.gold")

print("✅ Created namespaces: bronze, silver, gold")


[2/10] Creating Iceberg schemas...


IllegalArgumentException: Cannot initialize FileIO implementation org.apache.iceberg.aws.s3.S3FileIO: Cannot find constructor for interface org.apache.iceberg.io.FileIO
	Missing org.apache.iceberg.aws.s3.S3FileIO [java.lang.NoClassDefFoundError: software/amazon/awssdk/services/s3/model/S3Exception]

In [8]:
!pip install sql

  Preparing metadata (setup.py) ... done
  Created wheel for sql: filename=sql-2022.4.0-py3-none-any.whl size=4306 sha256=ddceb7ed61c2c130850dbdab40db8cd8cbe27bc0483007f58fdc8d1d95aab477
  Stored in directory: /home/jovyan/.cache/pip/wheels/a6/f1/62/be6faba20c8384c5766c0332c93841b610ab2602d6ae312bbf
Successfully built sql


In [33]:
# ============================================================
# STEP 1: CREATE SPARK SESSION (FIXED)
# ============================================================
print("\n[1/10] Creating Spark Session with Local JARs...")

# Inside the container, ./shared is mapped to /opt/shared-data
jar_path = "/opt/shared-data"
iceberg_runtime = f"{jar_path}/iceberg-spark-runtime-3.5_2.12-1.5.0.jar"
aws_bundle = f"{jar_path}/iceberg-aws-bundle-1.5.0.jar"

spark = SparkSession.builder \
    .appName("Retail ETL Pipeline - Iceberg REST") \
    .master("spark://spark-master:7077") \
    .config("spark.jars", f"{iceberg_runtime},{aws_bundle}") \
    .config("spark.executor.extraClassPath", f"{iceberg_runtime}:{aws_bundle}") \
    .config("spark.driver.extraClassPath", f"{iceberg_runtime}:{aws_bundle}") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.iceberg.catalog-impl", "org.apache.iceberg.rest.RESTCatalog") \
    .config("spark.sql.catalog.iceberg.uri", "http://iceberg-rest:8181") \
    .config("spark.sql.catalog.iceberg.warehouse", "s3://lakehouse/warehouse") \
    .config("spark.sql.catalog.iceberg.io-impl", "org.apache.iceberg.aws.s3.S3FileIO") \
    .config("spark.sql.catalog.iceberg.s3.endpoint", "http://minio:9000") \
    .config("spark.sql.catalog.iceberg.s3.path-style-access", "true") \
    .config("spark.sql.catalog.iceberg.s3.access-key-id", "minio") \
    .config("spark.sql.catalog.iceberg.s3.secret-access-key", "minio123") \
    .config("spark.sql.catalog.iceberg.s3.client.region", "us-east-1") \
    .getOrCreate()


[1/10] Creating Spark Session with Local JARs...


In [45]:
# Check if the session can actually see the S3 classes now
try:
    spark._jvm.software.amazon.awssdk.services.s3.model.S3Exception
    print("✅ Worker/Driver synchronization successful!")
except Exception as e:
    print(f"❌ Still missing: {e}")

✅ Worker/Driver synchronization successful!


In [46]:
# STEP 2: CREATE ICEBERG NAMESPACES
# ============================================================
print("\n[2/10] Creating Iceberg namespaces...")

try:
    spark.sql("CREATE NAMESPACE IF NOT EXISTS iceberg.bronze")
    print("✅ Created: iceberg.bronze")
except Exception as e:
    print(f"⚠️  bronze: {e}")

try:
    spark.sql("CREATE NAMESPACE IF NOT EXISTS iceberg.silver")
    print("✅ Created: iceberg.silver")
except Exception as e:
    print(f"⚠️  silver: {e}")

try:
    spark.sql("CREATE NAMESPACE IF NOT EXISTS iceberg.gold")
    print("✅ Created: iceberg.gold")
except Exception as e:
    print(f"⚠️  gold: {e}")

# List namespaces
print("\nAvailable namespaces in Iceberg:")
try:
    spark.sql("SHOW NAMESPACES IN iceberg").show()
except Exception as e:
    print(f"⚠️  {e}")


[2/10] Creating Iceberg namespaces...
⚠️  bronze: Cannot initialize FileIO implementation org.apache.iceberg.aws.s3.S3FileIO: Cannot find constructor for interface org.apache.iceberg.io.FileIO
	Missing org.apache.iceberg.aws.s3.S3FileIO [java.lang.NoClassDefFoundError: software/amazon/awssdk/services/s3/model/S3Exception]
⚠️  silver: Cannot initialize FileIO implementation org.apache.iceberg.aws.s3.S3FileIO: Cannot find constructor for interface org.apache.iceberg.io.FileIO
	Missing org.apache.iceberg.aws.s3.S3FileIO [java.lang.NoClassDefFoundError: software/amazon/awssdk/services/s3/model/S3Exception]
⚠️  gold: Cannot initialize FileIO implementation org.apache.iceberg.aws.s3.S3FileIO: Cannot find constructor for interface org.apache.iceberg.io.FileIO
	Missing org.apache.iceberg.aws.s3.S3FileIO [java.lang.NoClassDefFoundError: software/amazon/awssdk/services/s3/model/S3Exception]

Available namespaces in Iceberg:
⚠️  Cannot initialize FileIO implementation org.apache.iceberg.aws.s3.S

In [33]:
# STEP 3: LOAD RAW DATA
# ============================================================
print("\n[3/10] Loading raw data...")
df_raw = spark.read.csv("/opt/spark-data/online_retail_combined.csv",
                        header=True,
                        inferSchema=True)

row_count = df_raw.count()
print(f"✅ Loaded {row_count:,} rows")
df_raw.printSchema()


[3/10] Loading raw data...
✅ Loaded 1,067,371 rows
root
 |-- Invoice: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- Price: double (nullable = true)
 |-- Customer ID: double (nullable = true)
 |-- Country: string (nullable = true)
 |-- fiscal_year: string (nullable = true)



In [20]:
# STEP 4: DATA QUALITY CHECKS
# ============================================================
print("\n[4/10] Running data quality checks...")

null_counts = df_raw.select([
    count(when(col(c).isNull(), c)).alias(c) 
    for c in df_raw.columns
])
print("\n📊 Null value counts:")
null_counts.show()

total_rows = df_raw.count()
distinct_rows = df_raw.distinct().count()
print(f"\n📋 Total rows: {total_rows:,}")
print(f"📋 Distinct rows: {distinct_rows:,}")
print(f"📋 Duplicates: {total_rows - distinct_rows:,}")



[4/10] Running data quality checks...

📊 Null value counts:
+-------+---------+-----------+--------+-----------+-----+-----------+-------+-----------+
|Invoice|StockCode|Description|Quantity|InvoiceDate|Price|Customer ID|Country|fiscal_year|
+-------+---------+-----------+--------+-----------+-----+-----------+-------+-----------+
|      0|        0|       4382|       0|          0|    0|     243007|      0|          0|
+-------+---------+-----------+--------+-----------+-----+-----------+-------+-----------+


📋 Total rows: 1,067,371
📋 Distinct rows: 1,055,238
📋 Duplicates: 12,133


In [21]:
# STEP 5: DATA CLEANING
# ============================================================
print("\n[5/10] Cleaning data...")

df_cleaned = df_raw \
    .filter(col("Invoice").isNotNull()) \
    .filter(col("StockCode").isNotNull()) \
    .filter(col("Quantity") > 0) \
    .filter(col("Price") > 0) \
    .filter(col("Customer ID").isNotNull()) \
    .dropDuplicates()

print(f"✅ After cleaning: {df_cleaned.count():,} rows")
print(f"   Removed: {df_raw.count() - df_cleaned.count():,} rows")

# ============================================================
# STEP 6: DATA TRANSFORMATION
# ============================================================
print("\n[6/10] Transforming data...")

df_transformed = df_cleaned \
    .withColumn("InvoiceDate", to_timestamp(col("InvoiceDate"), "yyyy-MM-dd HH:mm:ss")) \
    .withColumn("TotalAmount", col("Quantity") * col("Price")) \
    .withColumn("Year", year(col("InvoiceDate"))) \
    .withColumn("Month", month(col("InvoiceDate"))) \
    .withColumn("DayOfWeek", dayofweek(col("InvoiceDate"))) \
    .withColumn("Hour", hour(col("InvoiceDate"))) \
    .withColumn("CustomerID", col("Customer ID").cast("string")) \
    .drop("Customer ID")

print("✅ Added columns: TotalAmount, Year, Month, DayOfWeek, Hour")



[5/10] Cleaning data...
✅ After cleaning: 793,609 rows
   Removed: 273,762 rows

[6/10] Transforming data...
✅ Added columns: TotalAmount, Year, Month, DayOfWeek, Hour


In [31]:
# ============================================================
# STEP 2: SET DEFAULT CATALOG & CREATE ICEBERG NAMESPACES
# ============================================================
print("\n[2/10] Setting Iceberg as default catalog and creating namespaces...")

# Check current catalog
try:
    current_catalog = spark.sql("SELECT current_catalog()").collect()[0][0]
    print(f"Current catalog: {current_catalog}")
except:
    print("Could not determine current catalog")

# Set iceberg as the current catalog
try:
    spark.sql("USE CATALOG iceberg")
    print("✅ Set current catalog to: iceberg")
    
    # Verify
    current_catalog = spark.sql("SELECT current_catalog()").collect()[0][0]
    print(f"Verified current catalog: {current_catalog}")
except Exception as e:
    print(f"⚠️  Error setting catalog: {e}")
    import traceback
    traceback.print_exc()

# Create namespaces
print("\nCreating namespaces...")
namespaces = ["bronze", "silver", "gold"]
for ns in namespaces:
    try:
        spark.sql(f"CREATE NAMESPACE IF NOT EXISTS {ns}")
        print(f"✅ Created: {ns}")
    except Exception as e:
        print(f"⚠️  {ns}: {e}")
        import traceback
        traceback.print_exc()

# List namespaces
print("\nAvailable namespaces in Iceberg:")
try:
    spark.sql("SHOW NAMESPACES").show()
except Exception as e:
    print(f"⚠️  {e}")


[2/10] Setting Iceberg as default catalog and creating namespaces...
Current catalog: spark_catalog
⚠️  Error setting catalog: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'iceberg': extra input 'iceberg'.(line 1, pos 12)

== SQL ==
USE CATALOG iceberg
------------^^^


Creating namespaces...
✅ Created: bronze
✅ Created: silver
✅ Created: gold

Available namespaces in Iceberg:
+---------+
|namespace|
+---------+
|   bronze|
|  default|
|     gold|
|   silver|
+---------+



Traceback (most recent call last):
  File "/tmp/ipykernel_1891/3170376295.py", line 15, in <module>
    spark.sql("USE CATALOG iceberg")
  File "/usr/local/spark/python/pyspark/sql/session.py", line 1631, in sql
    return DataFrame(self._jsparkSession.sql(sqlQuery, litArgs), self)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
                   ^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/pyspark/errors/exceptions/captured.py", line 185, in deco
    raise converted from None
pyspark.errors.exceptions.captured.ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'iceberg': extra input 'iceberg'.(line 1, pos 12)

== SQL ==
USE CATALOG iceberg
------------^^^



In [1]:
from pyspark.sql import SparkSession

# Stop any existing session
try:
    spark.stop()
except:
    pass

# Create Spark session with Polaris catalog
spark = (
    SparkSession.builder
    .appName("Retail-Lakehouse-Polaris")
    
    # Connect to Spark cluster
    .master("spark://spark-master:7077")
    .config("spark.driver.host", "jupyter")
    .config("spark.driver.bindAddress", "0.0.0.0")
    
    # Iceberg Extensions (REQUIRED)
    .config("spark.sql.extensions", 
            "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    
    # Polaris REST Catalog
    .config("spark.sql.catalog.polaris", 
            "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.polaris.catalog-impl", 
            "org.apache.iceberg.rest.RESTCatalog")
    .config("spark.sql.catalog.polaris.uri", 
            "http://polaris:8181")
    .config("spark.sql.catalog.polaris.credential", 
            "8bc42068486af1a8:d3eed0a93918a2fa99850097d43b8fd0")  # ← NEW CREDENTIAL
    .config("spark.sql.catalog.polaris.warehouse", 
            "lakehouse")
    
    # S3/MinIO for data storage
    .config("spark.sql.catalog.polaris.io-impl", 
            "org.apache.iceberg.aws.s3.S3FileIO")
    .config("spark.sql.catalog.polaris.s3.endpoint", 
            "http://minio:9000")
    .config("spark.sql.catalog.polaris.s3.access-key-id", 
            "minio")
    .config("spark.sql.catalog.polaris.s3.secret-access-key", 
            "minio123")
    .config("spark.sql.catalog.polaris.s3.path-style-access", 
            "true")
    
    .getOrCreate()
)

print("✅ Spark session created successfully!")
print(f"Spark Version: {spark.version}")
print(f"Master: {spark.sparkContext.master}")

✅ Spark session created successfully!
Spark Version: 3.5.0
Master: spark://spark-master:7077


In [2]:
# Test 1: Show available catalogs
print("📋 Available catalogs:")
spark.sql("SHOW CATALOGS").show()

📋 Available catalogs:
+-------------+
|      catalog|
+-------------+
|spark_catalog|
+-------------+



In [3]:
# Check Spark configuration
print("Current Spark configs related to Iceberg/Polaris:")
for conf in spark.sparkContext.getConf().getAll():
    if 'iceberg' in conf[0].lower() or 'polaris' in conf[0].lower() or 'catalog' in conf[0].lower():
        print(f"{conf[0]} = {conf[1]}")

Current Spark configs related to Iceberg/Polaris:
spark.sql.catalog.polaris.io-impl = org.apache.iceberg.aws.s3.S3FileIO
spark.sql.catalog.polaris.s3.path-style-access = true
spark.sql.catalog.polaris.warehouse = lakehouse
spark.sql.catalog.polaris.catalog-impl = org.apache.iceberg.rest.RESTCatalog
spark.sql.catalog.polaris = org.apache.iceberg.spark.SparkCatalog
spark.sql.catalog.polaris.s3.access-key-id = minio
spark.sql.catalog.polaris.s3.secret-access-key = minio123
spark.sql.catalog.polaris.uri = http://polaris:8181
spark.sql.catalog.polaris.credential = 8bc42068486af1a8:d3eed0a93918a2fa99850097d43b8fd0
spark.sql.catalog.polaris.s3.endpoint = http://minio:9000


In [5]:
# Check JARs (fixed version)
import os

# Method 1: Check classpath
print("Checking classpath for Iceberg JARs:")
classpath = spark.sparkContext._jvm.System.getProperty("java.class.path")
for entry in classpath.split(":"):
    if 'iceberg' in entry.lower():
        print(f"✅ Found: {entry}")

# Method 2: Check if Iceberg classes are available
print("\n" + "="*50)
print("Checking if Iceberg classes can be loaded:")
try:
    spark.sparkContext._jvm.org.apache.iceberg.spark.SparkCatalog
    print("✅ SparkCatalog class found!")
except Exception as e:
    print(f"❌ SparkCatalog class NOT found: {e}")

try:
    spark.sparkContext._jvm.org.apache.iceberg.rest.RESTCatalog
    print("✅ RESTCatalog class found!")
except Exception as e:
    print(f"❌ RESTCatalog class NOT found: {e}")

Checking classpath for Iceberg JARs:

Checking if Iceberg classes can be loaded:
✅ SparkCatalog class found!
✅ RESTCatalog class found!


In [37]:
# ============================================================
# STEP 7: SAVE TO ICEBERG LAKEHOUSE (YOUR CATALOG!)
# ============================================================
print("\n[8/8] Saving to Iceberg Lakehouse...")

try:
    # BRONZE: Raw cleaned data
    df_transformed.writeTo("demo.bronze.retail_transactions") \
        .using("iceberg") \
        .partitionedBy("Year", "Month") \
        .createOrReplace()
    print("✅ BRONZE: demo.bronze.retail_transactions")

    # SILVER: Customer metrics  
    df_customer_metrics.writeTo("demo.silver.customer_metrics") \
        .using("iceberg") \
        .createOrReplace()
    print("✅ SILVER: demo.silver.customer_metrics")

    # SILVER: Product metrics
    df_product_metrics.writeTo("demo.silver.product_metrics") \
        .using("iceberg") \
        .createOrReplace()
    print("✅ SILVER: demo.silver.product_metrics")

    # SILVER: RFM analysis
    df_rfm_scored.writeTo("demo.silver.rfm_analysis") \
        .using("iceberg") \
        .createOrReplace()
    print("✅ SILVER: demo.silver.rfm_analysis")

    # GOLD: Top customers by RFM score
    df_top_customers = df_rfm_scored.filter(col("RFM_Score") >= "555") \
        .select("CustomerID", "Recency", "Frequency", "Monetary", "RFM_Score")
    
    df_top_customers.writeTo("demo.gold.top_customers") \
        .using("iceberg") \
        .createOrReplace()
    print("✅ GOLD: demo.gold.top_customers")

except Exception as e:
    print(f"⚠️  Iceberg save error: {e}")



[8/8] Saving to Iceberg Lakehouse...
⚠️  Iceberg save error: An error occurred while calling o1786.createOrReplace.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:499)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:480)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.inv

In [6]:
from pyspark.sql import SparkSession

# ------------------------------------------------------------
# Stop any existing Spark session
# ------------------------------------------------------------
try:
    spark.stop()
except:
    pass

# ------------------------------------------------------------
# Create Spark Session (ICEBERG + REST)
# ------------------------------------------------------------
spark = (
    SparkSession.builder
    .appName("Retail-Lakehouse-Iceberg")

    # ---- Spark cluster ----
    .master("spark://spark-master:7077")
    .config("spark.driver.host", "jupyter")
    .config("spark.driver.bindAddress", "0.0.0.0")

    # ---- Iceberg Spark Extensions (MANDATORY) ----
    .config(
        "spark.sql.extensions",
        "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions"
    )

    # =========================================================
    # ICEBERG REST CATALOG (catalog name MUST be "iceberg")
    # =========================================================
    .config(
        "spark.sql.catalog.iceberg",
        "org.apache.iceberg.spark.SparkCatalog"
    )
    .config(
        "spark.sql.catalog.iceberg.catalog-impl",
        "org.apache.iceberg.rest.RESTCatalog"
    )
    .config(
        "spark.sql.catalog.iceberg.uri",
        "http://rest:8181"
    )

    .getOrCreate()
)

print("✅ Spark started successfully")

# ------------------------------------------------------------
# Validate
# ------------------------------------------------------------
print("\n📋 Available catalogs:")
spark.sql("SHOW CATALOGS").show()

print("\n📂 Namespaces in iceberg catalog:")
spark.sql("SHOW NAMESPACES IN iceberg").show()


Py4JJavaError: An error occurred while calling None.org.apache.spark.sql.SparkSession.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:499)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:480)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorComm

In [ ]:
spark.sql("SHOW CATALOGS").show()


In [49]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

# Step 2: Load the data using Spark
print("\n[1/8] Loading raw data...")
df_raw = spark.read.csv("/opt/spark-data/online_retail_combined.csv",
                        header=True,
                        inferSchema=True)

row_count = df_raw.count()
print(f"✅ Loaded {row_count:,} rows")
df_raw.printSchema()


# STEP 1: DATA QUALITY CHECKS
# ============================================================
print("\n[2/8] Running data quality checks...")

# Check for nulls in critical columns
null_counts = df_raw.select([
    count(when(col(c).isNull(), c)).alias(c) 
    for c in df_raw.columns
])
print("\n📊 Null value counts:")
null_counts.show()

# Check for duplicates
total_rows = df_raw.count()
distinct_rows = df_raw.distinct().count()
print(f"\n📋 Total rows: {total_rows:,}")
print(f"📋 Distinct rows: {distinct_rows:,}")
print(f"📋 Duplicates: {total_rows - distinct_rows:,}")

# ============================================================
# STEP 2: DATA CLEANING
# ============================================================
print("\n[3/8] Cleaning data...")

df_cleaned = df_raw \
    .filter(col("Invoice").isNotNull()) \
    .filter(col("StockCode").isNotNull()) \
    .filter(col("Quantity") > 0) \
    .filter(col("Price") > 0) \
    .filter(col("Customer ID").isNotNull()) \
    .dropDuplicates()

print(f"✅ After cleaning: {df_cleaned.count():,} rows")
print(f"   Removed: {df_raw.count() - df_cleaned.count():,} rows")

# ============================================================
# STEP 3: DATA TRANSFORMATION
# ============================================================
print("\n[4/8] Transforming data...")

df_transformed = df_cleaned \
    .withColumn("InvoiceDate", to_timestamp(col("InvoiceDate"), "yyyy-MM-dd HH:mm:ss")) \
    .withColumn("TotalAmount", col("Quantity") * col("Price")) \
    .withColumn("Year", year(col("InvoiceDate"))) \
    .withColumn("Month", month(col("InvoiceDate"))) \
    .withColumn("DayOfWeek", dayofweek(col("InvoiceDate"))) \
    .withColumn("Hour", hour(col("InvoiceDate"))) \
    .withColumn("CustomerID", col("Customer ID").cast("string")) \
    .drop("Customer ID")

print("✅ Added columns: TotalAmount, Year, Month, DayOfWeek, Hour")

# ============================================================
# STEP 4: CREATE BUSINESS METRICS
# ============================================================
print("\n[5/8] Creating business metrics...")

# Customer-level aggregations
df_customer_metrics = df_transformed.groupBy("CustomerID") \
    .agg(
        count("Invoice").alias("TotalOrders"),
        sum("TotalAmount").alias("TotalSpent"),
        avg("TotalAmount").alias("AvgOrderValue"),
        countDistinct("Invoice").alias("UniqueInvoices"),
        min("InvoiceDate").alias("FirstPurchaseDate"),
        max("InvoiceDate").alias("LastPurchaseDate")
    ) \
    .withColumn("DaysSinceFirstPurchase", 
                datediff(col("LastPurchaseDate"), col("FirstPurchaseDate")))

print("✅ Customer metrics created")
df_customer_metrics.show(5)

# Product-level aggregations
df_product_metrics = df_transformed.groupBy("StockCode", "Description") \
    .agg(
        sum("Quantity").alias("TotalQuantitySold"),
        sum("TotalAmount").alias("TotalRevenue"),
        countDistinct("CustomerID").alias("UniqueCustomers"),
        avg("Price").alias("AvgPrice")
    ) \
    .orderBy(desc("TotalRevenue"))

print("\n✅ Product metrics created")
df_product_metrics.show(5)

# ============================================================
# STEP 5: FEATURE ENGINEERING
# ============================================================
print("\n[6/8] Feature engineering...")

# RFM Analysis (Recency, Frequency, Monetary)
from datetime import datetime
max_date = df_transformed.select(max("InvoiceDate")).collect()[0][0]

df_rfm = df_transformed.groupBy("CustomerID") \
    .agg(
        datediff(lit(max_date), max("InvoiceDate")).alias("Recency"),
        countDistinct("Invoice").alias("Frequency"),
        sum("TotalAmount").alias("Monetary")
    )

# Add RFM scores (1-5 scale)
rfm_quantiles = df_rfm.approxQuantile(["Recency", "Frequency", "Monetary"], 
                                       [0.2, 0.4, 0.6, 0.8], 0.01)

df_rfm_scored = df_rfm \
    .withColumn("R_Score", 
                when(col("Recency") <= rfm_quantiles[0][0], 5)
                .when(col("Recency") <= rfm_quantiles[0][1], 4)
                .when(col("Recency") <= rfm_quantiles[0][2], 3)
                .when(col("Recency") <= rfm_quantiles[0][3], 2)
                .otherwise(1)) \
    .withColumn("F_Score",
                when(col("Frequency") >= rfm_quantiles[1][3], 5)
                .when(col("Frequency") >= rfm_quantiles[1][2], 4)
                .when(col("Frequency") >= rfm_quantiles[1][1], 3)
                .when(col("Frequency") >= rfm_quantiles[1][0], 2)
                .otherwise(1)) \
    .withColumn("M_Score",
                when(col("Monetary") >= rfm_quantiles[2][3], 5)
                .when(col("Monetary") >= rfm_quantiles[2][2], 4)
                .when(col("Monetary") >= rfm_quantiles[2][1], 3)
                .when(col("Monetary") >= rfm_quantiles[2][0], 2)
                .otherwise(1)) \
    .withColumn("RFM_Score", 
                concat(col("R_Score"), col("F_Score"), col("M_Score")))

print("✅ RFM analysis complete")
df_rfm_scored.show(10)

# ============================================================
# STEP 6: OUTLIER DETECTION
# ============================================================
print("\n[7/8] Detecting outliers...")

# Find outliers using IQR method
quantiles = df_transformed.approxQuantile("TotalAmount", [0.25, 0.75], 0.01)
Q1, Q3 = quantiles[0], quantiles[1]
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_outliers = df_transformed.filter(
    (col("TotalAmount") < lower_bound) | 
    (col("TotalAmount") > upper_bound)
)

print(f"✅ Found {df_outliers.count():,} outlier transactions")
print(f"   Lower bound: ${lower_bound:.2f}")
print(f"   Upper bound: ${upper_bound:.2f}")

# ============================================================
# ============================================================
# STEP 7: SAVE TO ICEBERG LAKEHOUSE (YOUR CATALOG!)
# ============================================================
print("\n[8/8] Saving to Iceberg Lakehouse...")
# -------------------------
# BRONZE – FACT TABLE
# -------------------------
(
    df_transformed
    .writeTo("iceberg.bronze.retail_transactions")
    .using("iceberg")
    .partitionedBy("Year", "Month")
    .tableProperty("format-version", "2")
    .tableProperty("write.format.default", "parquet")
    .tableProperty("write.metadata.delete-after-commit.enabled", "true")
    .tableProperty("write.metadata.previous-versions-max", "5")
    .overwritePartitions()
)

print("✅ BRONZE table written (partition overwrite)")

# -------------------------
# SILVER – CUSTOMER METRICS
# -------------------------
(
    df_customer_metrics
    .writeTo("iceberg.silver.customer_metrics")
    .using("iceberg")
    .tableProperty("format-version", "2")
    .createOrReplace()
)

print("✅ SILVER customer_metrics written")

# -------------------------
# SILVER – PRODUCT METRICS
# -------------------------
(
    df_product_metrics
    .writeTo("iceberg.silver.product_metrics")
    .using("iceberg")
    .tableProperty("format-version", "2")
    .createOrReplace()
)

print("✅ SILVER product_metrics written")

# -------------------------
# SILVER – RFM ANALYSIS
# -------------------------
(
    df_rfm_scored
    .writeTo("iceberg.silver.rfm_analysis")
    .using("iceberg")
    .tableProperty("format-version", "2")
    .createOrReplace()
)

print("✅ SILVER rfm_analysis written")

# -------------------------
# GOLD – TOP CUSTOMERS
# -------------------------
df_top_customers = (
    df_rfm_scored
    .filter(col("RFM_Score") >= "555")
    .select("CustomerID", "Recency", "Frequency", "Monetary", "RFM_Score")
)

(
    df_top_customers
    .writeTo("iceberg.gold.top_customers")
    .using("iceberg")
    .tableProperty("format-version", "2")
    .createOrReplace()
)

print("✅ GOLD top_customers written")



[1/8] Loading raw data...
✅ Loaded 1,067,371 rows
root
 |-- Invoice: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- Price: double (nullable = true)
 |-- Customer ID: double (nullable = true)
 |-- Country: string (nullable = true)
 |-- fiscal_year: string (nullable = true)


[2/8] Running data quality checks...

📊 Null value counts:
+-------+---------+-----------+--------+-----------+-----+-----------+-------+-----------+
|Invoice|StockCode|Description|Quantity|InvoiceDate|Price|Customer ID|Country|fiscal_year|
+-------+---------+-----------+--------+-----------+-----+-----------+-------+-----------+
|      0|        0|       4382|       0|          0|    0|     243007|      0|          0|
+-------+---------+-----------+--------+-----------+-----+-----------+-------+-----------+


📋 Total rows: 1,067,371
📋 Distinct rows: 1,055,238
📋 D

IllegalArgumentException: Cannot initialize FileIO implementation org.apache.iceberg.aws.s3.S3FileIO: Cannot find constructor for interface org.apache.iceberg.io.FileIO
	Missing org.apache.iceberg.aws.s3.S3FileIO [java.lang.NoClassDefFoundError: software/amazon/awssdk/services/s3/model/S3Exception]

In [16]:
spark.sql("SHOW TABLES IN demo.bronze").show()
spark.sql("SELECT COUNT(*) FROM demo.bronze.retail_transactions").show()


+---------+-------------------+-----------+
|namespace|          tableName|isTemporary|
+---------+-------------------+-----------+
|   bronze|retail_transactions|      false|
+---------+-------------------+-----------+

+--------+
|count(1)|
+--------+
|  793609|
+--------+



In [55]:
spark.sql("SHOW CATALOGS").show()


+-------------+
|      catalog|
+-------------+
|spark_catalog|
+-------------+



In [54]:
spark.sql("SHOW NAMESPACES IN iceberg").show()


IllegalArgumentException: Cannot initialize FileIO implementation org.apache.iceberg.aws.s3.S3FileIO: Cannot find constructor for interface org.apache.iceberg.io.FileIO
	Missing org.apache.iceberg.aws.s3.S3FileIO [java.lang.NoClassDefFoundError: software/amazon/awssdk/services/s3/model/S3Exception]

In [53]:

# ============================================================
# STEP 7: SAVE TO ICEBERG LAKEHOUSE (YOUR CATALOG!)
# ============================================================

# SILVER – CUSTOMER METRICS
# -------------------------
(
    df_customer_metrics
    .writeTo("iceberg.silver.customer_metrics")
    .using("iceberg")
    .tableProperty("format-version", "2")
    .createOrReplace()
)

print("✅ SILVER customer_metrics written")

# -------------------------
# SILVER – PRODUCT METRICS
# -------------------------
(
    df_product_metrics
    .writeTo("iceberg.silver.product_metrics")
    .using("iceberg")
    .tableProperty("format-version", "2")
    .createOrReplace()
)

print("✅ SILVER product_metrics written")

# -------------------------
# SILVER – RFM ANALYSIS
# -------------------------
(
    df_rfm_scored
    .writeTo("iceberg.silver.rfm_analysis")
    .using("iceberg")
    .tableProperty("format-version", "2")
    .createOrReplace()
)

print("✅ SILVER rfm_analysis written")

# -------------------------
# GOLD – TOP CUSTOMERS
# -------------------------
df_top_customers = (
    df_rfm_scored
    .filter(col("RFM_Score") >= "555")
    .select("CustomerID", "Recency", "Frequency", "Monetary", "RFM_Score")
)

(
    df_top_customers
    .writeTo("iceberg.gold.top_customers")
    .using("iceberg")
    .tableProperty("format-version", "2")
    .createOrReplace()
)

print("✅ GOLD top_customers written")


IllegalArgumentException: Cannot initialize FileIO implementation org.apache.iceberg.aws.s3.S3FileIO: Cannot find constructor for interface org.apache.iceberg.io.FileIO
	Missing org.apache.iceberg.aws.s3.S3FileIO [java.lang.NoClassDefFoundError: software/amazon/awssdk/services/s3/model/S3Exception]

In [52]:
df_transformed.writeTo(
    "iceberg.bronze.retail_transactions"
).using("iceberg") \
 .partitionedBy("Year", "Month") \
 .createOrReplace()


IllegalArgumentException: Cannot initialize FileIO implementation org.apache.iceberg.aws.s3.S3FileIO: Cannot find constructor for interface org.apache.iceberg.io.FileIO
	Missing org.apache.iceberg.aws.s3.S3FileIO [java.lang.NoClassDefFoundError: software/amazon/awssdk/services/s3/model/S3Exception]

In [13]:
spark.sql("SHOW CATALOGS").show()


+-------------+
|      catalog|
+-------------+
|spark_catalog|
+-------------+



In [14]:
spark.sql("SHOW NAMESPACES IN iceberg").show()


Py4JJavaError: An error occurred while calling o276.sql.
: org.apache.spark.SparkException: Cannot find catalog plugin class for catalog 'iceberg': org.apache.iceberg.spark.SparkCatalog.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.catalogPluginClassNotFoundForCatalogError(QueryExecutionErrors.scala:1925)
	at org.apache.spark.sql.connector.catalog.Catalogs$.load(Catalogs.scala:70)
	at org.apache.spark.sql.connector.catalog.CatalogManager.$anonfun$catalog$1(CatalogManager.scala:53)
	at scala.collection.mutable.HashMap.getOrElseUpdate(HashMap.scala:86)
	at org.apache.spark.sql.connector.catalog.CatalogManager.catalog(CatalogManager.scala:53)
	at org.apache.spark.sql.connector.catalog.LookupCatalog$CatalogAndNamespace$.unapply(LookupCatalog.scala:86)
	at org.apache.spark.sql.catalyst.analysis.ResolveCatalogs$$anonfun$apply$1.applyOrElse(ResolveCatalogs.scala:51)
	at org.apache.spark.sql.catalyst.analysis.ResolveCatalogs$$anonfun$apply$1.applyOrElse(ResolveCatalogs.scala:30)
	at o

In [6]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS iceberg.bronze")
spark.sql("CREATE NAMESPACE IF NOT EXISTS iceberg.silver")
spark.sql("CREATE NAMESPACE IF NOT EXISTS iceberg.gold")


DataFrame[]

In [18]:
spark.sql("SHOW TABLES IN iceberg.silver").show()


+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
+---------+---------+-----------+



In [19]:
spark.sql("""
CREATE TABLE iceberg.silver._debug_test (
  id INT,
  name STRING
)
USING iceberg
""")


Py4JJavaError: An error occurred while calling o62.sql.
: org.apache.iceberg.exceptions.ServiceFailureException: Server error: SdkClientException: Received an UnknownHostException when attempting to interact with a service. See cause for the exact endpoint that is failing to resolve. If this is happening on an endpoint that previously worked, there may be a network connectivity issue or your DNS cache could be storing endpoints for too long.
	at org.apache.iceberg.rest.ErrorHandlers$DefaultErrorHandler.accept(ErrorHandlers.java:217)
	at org.apache.iceberg.rest.ErrorHandlers$TableErrorHandler.accept(ErrorHandlers.java:118)
	at org.apache.iceberg.rest.ErrorHandlers$TableErrorHandler.accept(ErrorHandlers.java:102)
	at org.apache.iceberg.rest.HTTPClient.throwFailure(HTTPClient.java:201)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:313)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:252)
	at org.apache.iceberg.rest.HTTPClient.post(HTTPClient.java:358)
	at o

In [5]:
spark = SparkSession.builder \
    .appName("Phase3") \
    .master("spark://spark-master:7077") \
    .config("spark.driver.host", "jupyter-iceberg") \
    .config("spark.sql.catalog.iceberg.type", "rest") \
    .config("spark.sql.catalog.iceberg.uri", "http://iceberg-rest:8181") \
    .config("spark.sql.catalog.iceberg.warehouse", "s3a://lakehouse/warehouse") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minio") \
    .config("spark.hadoop.fs.s3a.secret.key", "minio123") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .getOrCreate()

spark.sql("SHOW CATALOGS").show()  # ✅ iceberg appears!


+-------------+
|      catalog|
+-------------+
|spark_catalog|
+-------------+



In [37]:
spark.stop()

AttributeError: 'Builder' object has no attribute 'stop'

In [1]:
spark = (
    SparkSession.builder
    .appName("Retail-Lakehouse-Iceberg")
    .master("spark://spark-master:7077")

    .config(
        "spark.sql.extensions",
        "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions"
    )

    .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.iceberg.type", "rest")
    .config("spark.sql.catalog.iceberg.uri", "http://iceberg-rest:8181")
    .config("spark.sql.catalog.iceberg.warehouse", "s3://lakehouse/warehouse")

    .getOrCreate()
)


NameError: name 'SparkSession' is not defined

In [7]:
spark.stop()

In [4]:
from pyspark.sql import SparkSession

try:
    spark.stop()
except:
    pass

spark = (
    SparkSession.builder
    .appName("Retail-Lakehouse-Polaris")
    .master("spark://spark-master:7077")

    .config("spark.driver.host", "jupyter")
    .config("spark.driver.bindAddress", "0.0.0.0")

    # REQUIRED
    .config(
        "spark.sql.extensions",
        "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions"
    )

    # Polaris REST catalog
    .config("spark.sql.catalog.polaris", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.polaris.catalog-impl", "org.apache.iceberg.rest.RESTCatalog")
    .config("spark.sql.catalog.polaris.uri", "http://polaris:8181")
    .config("spark.sql.catalog.polaris.warehouse", "s3://lakehouse/warehouse")

    # MinIO (S3)
    .config("spark.sql.catalog.polaris.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    .config("spark.sql.catalog.polaris.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.polaris.s3.access-key-id", "minio")
    .config("spark.sql.catalog.polaris.s3.secret-access-key", "minio123")
    .config("spark.sql.catalog.polaris.s3.path-style-access", "true")

    .getOrCreate()
)

spark.sql("SHOW CATALOGS").show()


+-------------+
|      catalog|
+-------------+
|spark_catalog|
+-------------+



In [4]:
from pyspark.sql import SparkSession

try:
    spark.stop()
except:
    pass

spark = (
    SparkSession.builder
    .appName("Retail-Lakehouse-Nessie")
    .master("spark://spark-master:7077")

    # Docker-safe networking
    .config("spark.driver.host", "jupyter")
    .config("spark.driver.bindAddress", "0.0.0.0")

    # Extensions (Iceberg + Nessie)
    .config(
        "spark.sql.extensions",
        "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,"
        "org.projectnessie.spark.extensions.NessieSparkSessionExtensions"
    )

    # Nessie catalog
    .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .config("spark.sql.catalog.nessie.uri", "http://nessie:19120/api/v1")
    .config("spark.sql.catalog.nessie.ref", "main")
    .config("spark.sql.catalog.nessie.warehouse", "s3a://lakehouse/warehouse")

    # S3 / MinIO
    .config("spark.sql.catalog.nessie.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    .config("spark.sql.catalog.nessie.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.nessie.s3.access-key-id", "minio")
    .config("spark.sql.catalog.nessie.s3.secret-access-key", "minio123")
    .config("spark.sql.catalog.nessie.s3.path-style-access", "true")

    # Hadoop S3A
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", "minio")
    .config("spark.hadoop.fs.s3a.secret.key", "minio123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")

    .getOrCreate()
)


In [2]:
spark.conf.get("spark.sql.extensions")


'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions'

In [3]:
spark._jvm.java.lang.Class.forName(
    "org.apache.iceberg.spark.SparkCatalog"
)


JavaObject id=o57

In [4]:
spark._jvm.java.lang.Class.forName(
    "org.apache.iceberg.rest.RESTCatalog"
)


JavaObject id=o58

In [5]:
spark.sql("SHOW CATALOGS").show(truncate=False)


+-------------+
|catalog      |
+-------------+
|spark_catalog|
+-------------+



In [4]:
spark version()
print(f"Spark Version: {spark.version}")
print(f"Spark Scala Version: {spark.sparkContext._gateway.jvm.scala.util.Properties.versionString()}")

SyntaxError: invalid syntax (3328262775.py, line 1)

In [6]:
spark.sql("SHOW CATALOGS").show()

+-------------+
|      catalog|
+-------------+
|spark_catalog|
+-------------+



In [4]:
spark.version


'3.5.0'

In [5]:
spark.sparkContext.master


'spark://spark-master:7077'

In [4]:
from pyspark.sql import SparkSession
import os

# ====== Configuration Variables ======
CATALOG_URI = "http://nessie:19120/api/v1"
WAREHOUSE = "s3a://lakehouse/warehouse"
MINIO_ENDPOINT = "http://minio:9000"
MINIO_ACCESS_KEY = "minio"
MINIO_SECRET_KEY = "minio123"
AWS_REGION = "us-east-1"  # Required for AWS SDK

# ====== JAR Configuration ======
JARS_PATH = "/home/jovyan/spark-jars"

# Check if JARs are present
print("🔍 Checking for JAR files...")
if os.path.exists(JARS_PATH):
    jar_files = [f for f in os.listdir(JARS_PATH) if f.endswith('.jar')]
    if jar_files:
        print(f"✅ Found {len(jar_files)} JAR files:")
        for jar in sorted(jar_files):
            jar_path = os.path.join(JARS_PATH, jar)
            size_mb = os.path.getsize(jar_path) / (1024*1024)
            print(f"  • {jar} ({size_mb:.1f} MB)")
    else:
        print(f"⚠️  No JAR files found in {JARS_PATH}")
else:
    print(f"❌ Directory {JARS_PATH} does not exist!")

print("\n" + "="*60)

# Build the JARs string
jars_list = [
    f"{JARS_PATH}/iceberg-spark-runtime-3.5_2.12-1.5.2.jar",
    f"{JARS_PATH}/iceberg-aws-bundle-1.5.2.jar",
    f"{JARS_PATH}/nessie-spark-extensions-3.5_2.12-0.84.0.jar"
]
jars_string = ",".join(jars_list)

print("🚀 Creating Spark Session with full AWS configuration...")
print("="*60)

# ====== Build Spark Session with AWS Region ======
try:
    spark = (
        SparkSession.builder
        .appName("Retail-Lakehouse-Iceberg-Nessie")
        .master("spark://spark-master:7077")
        
        # Load JARs
        .config("spark.jars", jars_string)
        .config("spark.driver.extraClassPath", f"{JARS_PATH}/*")
        .config("spark.executor.extraClassPath", f"{JARS_PATH}/*")
        
        # Enable Iceberg + Nessie SQL extensions
        .config("spark.sql.extensions",
                "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,"
                "org.projectnessie.spark.extensions.NessieSparkSessionExtensions")
        
        # Configure Nessie catalog
        .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
        .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
        .config("spark.sql.catalog.nessie.uri", CATALOG_URI)
        .config("spark.sql.catalog.nessie.ref", "main")
        .config("spark.sql.catalog.nessie.warehouse", WAREHOUSE)
        .config("spark.sql.catalog.nessie.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
        
        # CRITICAL: AWS Region configuration
        .config("spark.sql.catalog.nessie.client.region", AWS_REGION)
        
        # S3/MinIO configuration for Iceberg
        .config("spark.sql.catalog.nessie.s3.endpoint", MINIO_ENDPOINT)
        .config("spark.sql.catalog.nessie.s3.access-key-id", MINIO_ACCESS_KEY)
        .config("spark.sql.catalog.nessie.s3.secret-access-key", MINIO_SECRET_KEY)
        .config("spark.sql.catalog.nessie.s3.path-style-access", "true")
        
        # Hadoop S3A configuration
        .config("spark.hadoop.fs.s3a.endpoint", MINIO_ENDPOINT)
        .config("spark.hadoop.fs.s3a.access.key", MINIO_ACCESS_KEY)
        .config("spark.hadoop.fs.s3a.secret.key", MINIO_SECRET_KEY)
        .config("spark.hadoop.fs.s3a.path.style.access", "true")
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
        .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
        
        # CRITICAL: AWS SDK configuration (prevents region errors)
        .config("spark.hadoop.fs.s3a.aws.credentials.provider", 
                "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
        .config("spark.hadoop.com.amazonaws.services.s3.enableV4", "true")
        .config("spark.hadoop.fs.s3a.endpoint.region", AWS_REGION)
        
        .getOrCreate()
    )
    
    print("✅ Spark session created successfully!")
    print("="*60)
    
except Exception as e:
    print(f"❌ Failed to create Spark session: {e}")
    raise

# ====== Test Catalogs ======
print("\n📋 Available Catalogs:")
print("="*60)
spark.sql("SHOW CATALOGS").show(truncate=False)

# ====== Create Test Namespace and Table ======
print("\n🗂️ Creating test namespace and table...")
print("="*60)

try:
    # Create namespace
    spark.sql("CREATE NAMESPACE IF NOT EXISTS nessie.retail_test")
    print("✅ Namespace 'retail_test' created")
    
    # Create table
    spark.sql("""
        CREATE TABLE IF NOT EXISTS nessie.retail_test.sample_products (
            product_id INT,
            product_name STRING,
            category STRING,
            price DECIMAL(10,2),
            created_at TIMESTAMP
        ) USING iceberg
        TBLPROPERTIES (
            'format-version'='2',
            'write.parquet.compression-codec'='snappy'
        )
    """)
    print("✅ Table 'sample_products' created")
    
    # Insert sample data
    print("\n📝 Inserting sample data...")
    spark.sql("""
        INSERT INTO nessie.retail_test.sample_products VALUES
        (1, 'Laptop', 'Electronics', 999.99, current_timestamp()),
        (2, 'Mouse', 'Electronics', 29.99, current_timestamp()),
        (3, 'Desk Chair', 'Furniture', 199.99, current_timestamp()),
        (4, 'Monitor', 'Electronics', 299.99, current_timestamp()),
        (5, 'Keyboard', 'Electronics', 79.99, current_timestamp())
    """)
    print("✅ Sample data inserted")
    
    # Query the table
    print("\n📊 Querying sample_products table:")
    print("="*60)
    spark.sql("SELECT * FROM nessie.retail_test.sample_products ORDER BY product_id").show(truncate=False)
    
    # Show count
    count_df = spark.sql("SELECT COUNT(*) as total_products FROM nessie.retail_test.sample_products")
    total = count_df.collect()[0]['total_products']
    print(f"\n📈 Total products: {total}")
    
    # Show aggregations
    print("\n💰 Products by Category:")
    print("="*60)
    spark.sql("""
        SELECT 
            category,
            COUNT(*) as product_count,
            ROUND(AVG(price), 2) as avg_price,
            ROUND(SUM(price), 2) as total_value
        FROM nessie.retail_test.sample_products
        GROUP BY category
        ORDER BY total_value DESC
    """).show(truncate=False)
    
    # Show table history (Iceberg feature)
    print("\n📜 Table History (Iceberg Time Travel):")
    print("="*60)
    spark.sql("SELECT * FROM nessie.retail_test.sample_products.history").show(truncate=False)
    
    # Show snapshots
    print("\n📸 Table Snapshots:")
    print("="*60)
    spark.sql("SELECT * FROM nessie.retail_test.sample_products.snapshots").show(truncate=False)
    
    print("\n" + "="*60)
    print("🎉 SUCCESS! Your Retail Lakehouse is FULLY OPERATIONAL!")
    print("="*60)
    print("\n✅ What's working:")
    print("  • ✓ Spark 3.5.0")
    print("  • ✓ Apache Iceberg 1.5.2 (with time travel!)")
    print("  • ✓ Project Nessie 0.84.0 (catalog with Git-like versioning)")
    print("  • ✓ MinIO S3-compatible storage")
    print("  • ✓ AWS SDK properly configured")
    print("  • ✓ Data successfully written and queried")
    print("\n🚀 Advanced Features Available:")
    print("  • Time Travel: Query historical data")
    print("  • Schema Evolution: Add/remove columns safely")
    print("  • Partition Evolution: Change partitioning without rewriting data")
    print("  • Hidden Partitioning: No partition predicates needed in queries")
    print("  • ACID Transactions: Full consistency guarantees")
    print("  • Nessie Branches: Experiment with data changes safely")
    print("\n🎯 Next Steps:")
    print("  1. Create your retail schema (customers, orders, inventory)")
    print("  2. Set up NiFi for automated data ingestion")
    print("  3. Configure Trino for interactive SQL queries")
    print("  4. Build Superset dashboards for visualization")
    print("  5. Explore Nessie branching for dev/staging/prod workflows")
    
except Exception as e:
    print(f"\n❌ Error during table operations: {e}")
    print("\n🔧 Troubleshooting:")
    print("1. Check Nessie logs: docker logs nessie")
    print("2. Check MinIO access: docker exec mc mc ls local/lakehouse")
    print("3. Verify network connectivity between containers")
    import traceback
    traceback.print_exc()
    raise

🔍 Checking for JAR files...
✅ Found 3 JAR files:
  • iceberg-aws-bundle-1.5.2.jar (29.1 MB)
  • iceberg-spark-runtime-3.5_2.12-1.5.2.jar (39.7 MB)
  • nessie-spark-extensions-3.5_2.12-0.84.0.jar (0.0 MB)

🚀 Creating Spark Session with full AWS configuration...
✅ Spark session created successfully!

📋 Available Catalogs:
+-------------+
|catalog      |
+-------------+
|nessie       |
|spark_catalog|
+-------------+


🗂️ Creating test namespace and table...
✅ Namespace 'retail_test' created
✅ Table 'sample_products' created

📝 Inserting sample data...
✅ Sample data inserted

📊 Querying sample_products table:
+----------+------------+-----------+------+--------------------------+
|product_id|product_name|category   |price |created_at                |
+----------+------------+-----------+------+--------------------------+
|1         |Laptop      |Electronics|999.99|2026-01-28 16:18:52.080288|
|1         |Laptop      |Electronics|999.99|2026-01-28 16:17:58.793587|
|2         |Mouse       |E

In [2]:
spark.conf.get("spark.sql.extensions")


'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions'

In [3]:
spark.sql("SHOW CATALOGS").show(truncate=False)


+-------------+
|catalog      |
+-------------+
|nessie       |
|spark_catalog|
+-------------+



In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("Iceberg-Nessie-Retail")
    .master("spark://spark-master:7077")

    # =========================
    # Iceberg Extensions
    # =========================
    .config(
        "spark.sql.extensions",
        "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions"
    )

    # =========================
    # Nessie Catalog
    # =========================
    .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .config("spark.sql.catalog.nessie.uri", "http://nessie:19120/api/v1")
    .config("spark.sql.catalog.nessie.ref", "main")
    .config("spark.sql.catalog.nessie.authentication.type", "NONE")

    # =========================
    # Warehouse (MinIO)
    # =========================
    .config("spark.sql.catalog.nessie.warehouse", "s3a://warehouse/iceberg")

    # =========================
    # S3A → MinIO
    # =========================
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

    # =========================
    # 🚨 AWS SDK FIX (CRITICAL)
    # =========================
    .config("spark.hadoop.aws.region", "us-east-1")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider",
            "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")

    .getOrCreate()
)


In [5]:
spark.sql("SHOW CATALOGS").show()


+-------------+
|      catalog|
+-------------+
|       nessie|
|spark_catalog|
+-------------+



In [ ]:
spark.range(1, 10).count()


In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from datetime import datetime
import os

print("="*70)
print("🏪 RETAIL LAKEHOUSE - DATA PROCESSING PIPELINE")
print("="*70)

# ====== Configuration Variables ======
CATALOG_URI = "http://nessie:19120/api/v1"
WAREHOUSE = "s3a://lakehouse/warehouse"
MINIO_ENDPOINT = "http://minio:9000"
MINIO_ACCESS_KEY = "minio"
MINIO_SECRET_KEY = "minio123"
AWS_REGION = "us-east-1"
JARS_PATH = "/home/jovyan/spark-jars"

# ====== Build Spark Session ======
print("\n🔧 Creating Spark Session with Nessie catalog...")

jars_list = [
    f"{JARS_PATH}/iceberg-spark-runtime-3.5_2.12-1.5.2.jar",
    f"{JARS_PATH}/iceberg-aws-bundle-1.5.2.jar",
    f"{JARS_PATH}/nessie-spark-extensions-3.5_2.12-0.84.0.jar"
]
jars_string = ",".join(jars_list)

spark = (
    SparkSession.builder
    .appName("Retail-Data-Pipeline")
    .master("spark://spark-master:7077")
    .config("spark.jars", jars_string)
    .config("spark.driver.extraClassPath", f"{JARS_PATH}/*")
    .config("spark.executor.extraClassPath", f"{JARS_PATH}/*")
    .config("spark.sql.extensions",
            "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,"
            "org.projectnessie.spark.extensions.NessieSparkSessionExtensions")
    .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .config("spark.sql.catalog.nessie.uri", CATALOG_URI)
    .config("spark.sql.catalog.nessie.ref", "main")
    .config("spark.sql.catalog.nessie.warehouse", WAREHOUSE)
    .config("spark.sql.catalog.nessie.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    .config("spark.sql.catalog.nessie.client.region", AWS_REGION)
    .config("spark.sql.catalog.nessie.s3.endpoint", MINIO_ENDPOINT)
    .config("spark.sql.catalog.nessie.s3.access-key-id", MINIO_ACCESS_KEY)
    .config("spark.sql.catalog.nessie.s3.secret-access-key", MINIO_SECRET_KEY)
    .config("spark.sql.catalog.nessie.s3.path-style-access", "true")
    .config("spark.hadoop.fs.s3a.endpoint", MINIO_ENDPOINT)
    .config("spark.hadoop.fs.s3a.access.key", MINIO_ACCESS_KEY)
    .config("spark.hadoop.fs.s3a.secret.key", MINIO_SECRET_KEY)
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", 
            "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    .config("spark.hadoop.com.amazonaws.services.s3.enableV4", "true")
    .config("spark.hadoop.fs.s3a.endpoint.region", AWS_REGION)
    .getOrCreate()
)

print("✅ Spark session created with Nessie catalog!")

# ====== Create Namespaces ======
print("\n📁 Creating lakehouse namespaces (Bronze/Silver/Gold)...")
spark.sql("CREATE NAMESPACE IF NOT EXISTS nessie.bronze")
spark.sql("CREATE NAMESPACE IF NOT EXISTS nessie.silver")
spark.sql("CREATE NAMESPACE IF NOT EXISTS nessie.gold")
print("✅ Namespaces created: bronze, silver, gold")

# ====== STEP 1: LOAD RAW DATA ======
print("\n" + "="*70)
print("[1/8] 📥 LOADING RAW DATA")
print("="*70)

# Check if file exists
data_path = "/opt/spark-data/online_retail_combined.csv"
if os.path.exists(data_path):
    print(f"Loading from: {data_path}")
    df_raw = spark.read.csv(data_path, header=True, inferSchema=True)
    row_count = df_raw.count()
    print(f"✅ Loaded {row_count:,} rows")
    df_raw.printSchema()
else:
    print(f"⚠️  File not found: {data_path}")
    print("Creating sample data instead...")
    
    # Create sample data matching the expected schema
    schema = StructType([
        StructField("Invoice", StringType(), True),
        StructField("StockCode", StringType(), True),
        StructField("Description", StringType(), True),
        StructField("Quantity", IntegerType(), True),
        StructField("InvoiceDate", StringType(), True),
        StructField("Price", DoubleType(), True),
        StructField("Customer ID", StringType(), True),
        StructField("Country", StringType(), True)
    ])
    
    sample_data = [
        ("536365", "85123A", "WHITE HANGING HEART T-LIGHT HOLDER", 6, "2010-12-01 08:26:00", 2.55, "17850", "United Kingdom"),
        ("536365", "71053", "WHITE METAL LANTERN", 6, "2010-12-01 08:26:00", 3.39, "17850", "United Kingdom"),
        ("536365", "84406B", "CREAM CUPID HEARTS COAT HANGER", 8, "2010-12-01 08:26:00", 2.75, "17850", "United Kingdom"),
        ("536366", "22633", "HAND WARMER UNION JACK", 6, "2010-12-01 08:28:00", 1.85, "17850", "United Kingdom"),
        ("536367", "84879", "ASSORTED COLOUR BIRD ORNAMENT", 32, "2010-12-01 08:34:00", 1.69, "13047", "United Kingdom"),
        ("536367", "22745", "POPPY'S PLAYHOUSE BEDROOM", 6, "2010-12-01 08:34:00", 2.10, "13047", "United Kingdom"),
        ("536368", "22960", "JAM MAKING SET WITH JARS", 24, "2010-12-01 08:34:00", 1.45, "13047", "United Kingdom"),
        ("536369", "21756", "BATH BUILDING BLOCK WORD", 3, "2010-12-01 08:35:00", 5.95, "13047", "United Kingdom"),
        ("536370", "22086", "PAPER CHAIN KIT 50'S CHRISTMAS", 80, "2010-12-01 08:45:00", 2.55, "12583", "France"),
        ("536371", "22866", "HAND WARMER SCOTTY DOG DESIGN", 12, "2010-12-01 09:00:00", 2.10, "17850", "United Kingdom")
    ]
    
    df_raw = spark.createDataFrame(sample_data, schema)
    row_count = df_raw.count()
    print(f"✅ Created {row_count:,} sample rows")

# ====== STEP 2: DATA QUALITY CHECKS ======
print("\n" + "="*70)
print("[2/8] 🔍 DATA QUALITY CHECKS")
print("="*70)

# Check for nulls
null_counts = df_raw.select([
    count(when(col(c).isNull(), c)).alias(c) 
    for c in df_raw.columns
])
print("\n📊 Null value counts:")
null_counts.show()

# Check for duplicates
total_rows = df_raw.count()
distinct_rows = df_raw.distinct().count()
duplicates = total_rows - distinct_rows
print(f"\n📋 Total rows: {total_rows:,}")
print(f"📋 Distinct rows: {distinct_rows:,}")
print(f"📋 Duplicates: {duplicates:,}")

# ====== STEP 3: DATA CLEANING ======
print("\n" + "="*70)
print("[3/8] 🧹 DATA CLEANING")
print("="*70)

df_cleaned = df_raw \
    .filter(col("Invoice").isNotNull()) \
    .filter(col("StockCode").isNotNull()) \
    .filter(col("Quantity") > 0) \
    .filter(col("Price") > 0) \
    .filter(col("Customer ID").isNotNull()) \
    .dropDuplicates()

cleaned_count = df_cleaned.count()
removed = total_rows - cleaned_count
print(f"✅ After cleaning: {cleaned_count:,} rows")
print(f"   Removed: {removed:,} rows ({(removed/total_rows*100):.1f}%)")

# ====== STEP 4: DATA TRANSFORMATION ======
print("\n" + "="*70)
print("[4/8] ⚙️  DATA TRANSFORMATION")
print("="*70)

df_transformed = df_cleaned \
    .withColumn("InvoiceDate", to_timestamp(col("InvoiceDate"), "yyyy-MM-dd HH:mm:ss")) \
    .withColumn("TotalAmount", col("Quantity") * col("Price")) \
    .withColumn("Year", year(col("InvoiceDate"))) \
    .withColumn("Month", month(col("InvoiceDate"))) \
    .withColumn("DayOfWeek", dayofweek(col("InvoiceDate"))) \
    .withColumn("Hour", hour(col("InvoiceDate"))) \
    .withColumn("CustomerID", col("Customer ID").cast("string")) \
    .drop("Customer ID")

print("✅ Added columns:")
print("   • TotalAmount (Quantity × Price)")
print("   • Year, Month, DayOfWeek, Hour")
print("   • CustomerID (renamed from Customer ID)")

df_transformed.show(5, truncate=False)

# ====== STEP 5: CUSTOMER METRICS ======
print("\n" + "="*70)
print("[5/8] 👥 CUSTOMER METRICS")
print("="*70)

df_customer_metrics = df_transformed.groupBy("CustomerID") \
    .agg(
        count("Invoice").alias("TotalOrders"),
        sum("TotalAmount").alias("TotalSpent"),
        avg("TotalAmount").alias("AvgOrderValue"),
        countDistinct("Invoice").alias("UniqueInvoices"),
        min("InvoiceDate").alias("FirstPurchaseDate"),
        max("InvoiceDate").alias("LastPurchaseDate")
    ) \
    .withColumn("DaysSinceFirstPurchase", 
                datediff(col("LastPurchaseDate"), col("FirstPurchaseDate")))

print("✅ Customer metrics aggregated")
df_customer_metrics.orderBy(desc("TotalSpent")).show(5, truncate=False)

# ====== STEP 6: PRODUCT METRICS ======
print("\n" + "="*70)
print("[6/8] 📦 PRODUCT METRICS")
print("="*70)

df_product_metrics = df_transformed.groupBy("StockCode", "Description") \
    .agg(
        sum("Quantity").alias("TotalQuantitySold"),
        sum("TotalAmount").alias("TotalRevenue"),
        countDistinct("CustomerID").alias("UniqueCustomers"),
        avg("Price").alias("AvgPrice")
    ) \
    .orderBy(desc("TotalRevenue"))

print("✅ Product metrics calculated")
df_product_metrics.show(5, truncate=False)

# ====== STEP 7: RFM ANALYSIS ======
print("\n" + "="*70)
print("[7/8] 📊 RFM ANALYSIS (Recency, Frequency, Monetary)")
print("="*70)

# Get the latest date in dataset
max_date = df_transformed.select(max("InvoiceDate")).collect()[0][0]
print(f"Reference date for recency: {max_date}")

df_rfm = df_transformed.groupBy("CustomerID") \
    .agg(
        datediff(lit(max_date), max("InvoiceDate")).alias("Recency"),
        countDistinct("Invoice").alias("Frequency"),
        sum("TotalAmount").alias("Monetary")
    )

# Calculate RFM scores (1-5 scale)
rfm_quantiles = df_rfm.approxQuantile(["Recency", "Frequency", "Monetary"], 
                                       [0.2, 0.4, 0.6, 0.8], 0.01)

df_rfm_scored = df_rfm \
    .withColumn("R_Score", 
                when(col("Recency") <= rfm_quantiles[0][0], 5)
                .when(col("Recency") <= rfm_quantiles[0][1], 4)
                .when(col("Recency") <= rfm_quantiles[0][2], 3)
                .when(col("Recency") <= rfm_quantiles[0][3], 2)
                .otherwise(1)) \
    .withColumn("F_Score",
                when(col("Frequency") >= rfm_quantiles[1][3], 5)
                .when(col("Frequency") >= rfm_quantiles[1][2], 4)
                .when(col("Frequency") >= rfm_quantiles[1][1], 3)
                .when(col("Frequency") >= rfm_quantiles[1][0], 2)
                .otherwise(1)) \
    .withColumn("M_Score",
                when(col("Monetary") >= rfm_quantiles[2][3], 5)
                .when(col("Monetary") >= rfm_quantiles[2][2], 4)
                .when(col("Monetary") >= rfm_quantiles[2][1], 3)
                .when(col("Monetary") >= rfm_quantiles[2][0], 2)
                .otherwise(1)) \
    .withColumn("RFM_Score", 
                concat(col("R_Score"), col("F_Score"), col("M_Score"))) \
    .withColumn("CustomerSegment",
                when(col("RFM_Score") >= "444", "Champions")
                .when(col("RFM_Score") >= "344", "Loyal Customers")
                .when(col("RFM_Score") >= "244", "Potential Loyalists")
                .when(col("RFM_Score") >= "143", "Recent Customers")
                .when(col("RFM_Score") >= "111", "At Risk")
                .otherwise("Lost"))

print("✅ RFM analysis complete with customer segmentation")
df_rfm_scored.groupBy("CustomerSegment").count().orderBy(desc("count")).show()

# ====== STEP 8: SAVE TO ICEBERG LAKEHOUSE ======
print("\n" + "="*70)
print("[8/8] 💾 SAVING TO NESSIE ICEBERG LAKEHOUSE")
print("="*70)

# BRONZE LAYER - Raw cleaned data
print("\n🥉 Writing BRONZE layer (partitioned by Year, Month)...")
df_transformed.writeTo("nessie.bronze.retail_transactions") \
    .using("iceberg") \
    .partitionedBy("Year", "Month") \
    .tableProperty("format-version", "2") \
    .tableProperty("write.parquet.compression-codec", "snappy") \
    .createOrReplace()
print("✅ nessie.bronze.retail_transactions created")

# SILVER LAYER - Aggregated metrics
print("\n🥈 Writing SILVER layer (business metrics)...")

df_customer_metrics.writeTo("nessie.silver.customer_metrics") \
    .using("iceberg") \
    .tableProperty("format-version", "2") \
    .createOrReplace()
print("✅ nessie.silver.customer_metrics created")

df_product_metrics.writeTo("nessie.silver.product_metrics") \
    .using("iceberg") \
    .tableProperty("format-version", "2") \
    .createOrReplace()
print("✅ nessie.silver.product_metrics created")

df_rfm_scored.writeTo("nessie.silver.rfm_analysis") \
    .using("iceberg") \
    .tableProperty("format-version", "2") \
    .createOrReplace()
print("✅ nessie.silver.rfm_analysis created")

# GOLD LAYER - Top customers (for dashboards/ML)
print("\n🥇 Writing GOLD layer (curated datasets)...")

df_top_customers = df_rfm_scored \
    .filter(col("CustomerSegment").isin(["Champions", "Loyal Customers"])) \
    .select("CustomerID", "Recency", "Frequency", "Monetary", 
            "RFM_Score", "CustomerSegment")

df_top_customers.writeTo("nessie.gold.top_customers") \
    .using("iceberg") \
    .tableProperty("format-version", "2") \
    .createOrReplace()
print("✅ nessie.gold.top_customers created")

# ====== VERIFICATION ======
print("\n" + "="*70)
print("✅ PIPELINE COMPLETE - VERIFYING RESULTS")
print("="*70)

# Show all tables
print("\n📋 Tables created in Nessie catalog:")
spark.sql("SHOW TABLES IN nessie.bronze").show(truncate=False)
spark.sql("SHOW TABLES IN nessie.silver").show(truncate=False)
spark.sql("SHOW TABLES IN nessie.gold").show(truncate=False)

# Show record counts
print("\n📊 Record counts:")
bronze_count = spark.sql("SELECT COUNT(*) as count FROM nessie.bronze.retail_transactions").collect()[0]['count']
print(f"  • Bronze (transactions): {bronze_count:,}")

customer_count = spark.sql("SELECT COUNT(*) as count FROM nessie.silver.customer_metrics").collect()[0]['count']
print(f"  • Silver (customers): {customer_count:,}")

product_count = spark.sql("SELECT COUNT(*) as count FROM nessie.silver.product_metrics").collect()[0]['count']
print(f"  • Silver (products): {product_count:,}")

rfm_count = spark.sql("SELECT COUNT(*) as count FROM nessie.silver.rfm_analysis").collect()[0]['count']
print(f"  • Silver (RFM): {rfm_count:,}")

gold_count = spark.sql("SELECT COUNT(*) as count FROM nessie.gold.top_customers").collect()[0]['count']
print(f"  • Gold (top customers): {gold_count:,}")

# Sample queries
print("\n🔍 Sample query - Top 5 products by revenue:")
spark.sql("""
    SELECT 
        StockCode,
        Description,
        TotalRevenue,
        TotalQuantitySold,
        UniqueCustomers
    FROM nessie.silver.product_metrics
    ORDER BY TotalRevenue DESC
    LIMIT 5
""").show(truncate=False)

print("\n🔍 Customer segmentation distribution:")
spark.sql("""
    SELECT 
        CustomerSegment,
        COUNT(*) as CustomerCount,
        ROUND(SUM(Monetary), 2) as TotalRevenue,
        ROUND(AVG(Monetary), 2) as AvgRevenue
    FROM nessie.silver.rfm_analysis
    GROUP BY CustomerSegment
    ORDER BY TotalRevenue DESC
""").show(truncate=False)

print("\n" + "="*70)
print("🎉 RETAIL LAKEHOUSE PIPELINE SUCCESSFULLY COMPLETED!")
print("="*70)
print("\n✅ What was created:")
print("  • Bronze layer: Raw transaction data (partitioned by Year/Month)")
print("  • Silver layer: Customer, product, and RFM metrics")
print("  • Gold layer: Top customers for analytics")
print("\n📊 Lakehouse Features Available:")
print("  • Time Travel: Query historical data snapshots")
print("  • Schema Evolution: Add columns without breaking queries")
print("  • ACID Transactions: Full consistency guarantees")
print("  • Partition Evolution: Change partitioning strategy")
print("  • Nessie Branching: Test changes in dev branches")
print("\n🎯 Next Steps:")
print("  1. Query your data: spark.sql('SELECT * FROM nessie.silver.rfm_analysis')")
print("  2. Create Trino views for SQL access")
print("  3. Build Superset dashboards")
print("  4. Explore Nessie branches for experimentation")
print("  5. Implement ML models on RFM segments")

🏪 RETAIL LAKEHOUSE - DATA PROCESSING PIPELINE

🔧 Creating Spark Session with Nessie catalog...
✅ Spark session created with Nessie catalog!

📁 Creating lakehouse namespaces (Bronze/Silver/Gold)...
✅ Namespaces created: bronze, silver, gold

[1/8] 📥 LOADING RAW DATA
Loading from: /opt/spark-data/online_retail_combined.csv
✅ Loaded 1,067,371 rows
root
 |-- Invoice: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- Price: double (nullable = true)
 |-- Customer ID: double (nullable = true)
 |-- Country: string (nullable = true)
 |-- fiscal_year: string (nullable = true)


[2/8] 🔍 DATA QUALITY CHECKS

📊 Null value counts:
+-------+---------+-----------+--------+-----------+-----+-----------+-------+-----------+
|Invoice|StockCode|Description|Quantity|InvoiceDate|Price|Customer ID|Country|fiscal_year|
+-------+---------+-----------+--------+--